In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import traceback

from tqdm import tqdm
import pandas as pd



from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [75]:
!{sys.executable} -m pip install -r requirements.txt

  Using cached pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 15.3 MB 763 kB/s eta 0:00:01


In [12]:
logger({'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting"}},
       i=6033)

{'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting"}}


In [14]:
class Args():
    def __init__(self, term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
        
        self.term=term
        self.location=location
        self.streets_path=streets_path
        self.data_path=data_path
        self.iteration=iteration
        
    def init_parsearges(self):
        ap = argparse.ArgumentParser()
        
        # Argument of insightface
        ap.add_argument("--term", default=self.term,
                        type=str, help='Search term')
        
        ap.add_argument("--location", default=self.location,
                        type=str, help='Search location')
        ap.add_argument("--streets_path", default=self.streets_path,
                        type=str, help='path to names of the streets')
        ap.add_argument("--data_path", default=self.data_path,
                        type=str, help='path to save data')
        
        ap.add_argument("--iteration", default=self.iteration,
                         type=int, help='starting iteration')

        args = ap.parse_args()
        
        return args


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY= 'cwhlm9INXDobbgkfHlDhVVVVrncPHYNw9DoTcVLXuHkeCcGVVj7F7iPgMqaeW2JH5fKS24dRBQJoIkKJfLLgpJCGbPLmMuHSIqHsFsyyggXyzcohMWKSajdxGOHfX3Yx' 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'restaurants'
DEFAULT_LOCATION = 'Philadelphia, Pa'
SEARCH_LIMIT = 50


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

#     print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def logger(response,i):
    error=response['error']['code']
    description=response['error']['description']
    with open('error.log', 'a') as f:
        string=f'iteration={str(i)}\n    Error: "{error}"\n    Description: "{description}"\n'
        f.write(string)
        f.close()
    print(error)
    
def query_api(term, location, i):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        logger(response,i)
        return []

#     business_id = businesses[0]['id']

#     print(u'{0} businesses found, querying business info ' \
#         'for the top result "{1}" ...'.format(
#             len(businesses), business_id))
#     response = get_business(API_KEY, business_id)

#     data = [get_business(API_KEY, business['id']) for business in businesses ]
#     print(u'Result for business "{0}" found:'.format(business_id))
#     pprint.pprint(response, indent=2)
    return businesses

def read_street(path='street_name.txt'):
    with open(path, 'r') as f:
        streets = f.readlines()
    return [street.replace("\n","") for street in streets]

def read_data(path='data.pkl'):
    return pd.read_pickle(path)

def main(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0):
    
#     parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')

#     input_values = parser.parse_args()

    try:

    	streets = read_street(path=streets_path)
    	data=[] if iteration==0 else read_data(data_path).to_dict(orient='records')
    	for (i, street) in tqdm(enumerate(streets)):
        	if i>=iteration:
        		address = street +' '+ location
        		data.extend(query_api(term, address,i))
        	else:
        		pass
        
    	print("-------------------SAVING DATA---------------------")
    	df = pd.DataFrame(data)
    	df.to_pickle(data_path)
    
    	return df

    except:
        df = pd.DataFrame(data)
        print("===========SAVING DATA=============")
        df.to_pickle(data_path)
        traceback.print_exc()
        sys.exit()

In [ ]:
args=Args(term='restaurants', location='Philadelphia, Pa', streets_path='street_name.txt', data_path= 'data.pkl',iteration=0)
# args=args.init_parsearges()
b=main(term=args.term, location=args.location, streets_path=args.street_path, data_path= args.data_path,iteration=args.iteration)

# Data process

In [20]:
def foo(x):
#     print(x)
    dic={}
    for i in x:
#         print(i, i.keys())
        for key in i.keys():
            dic.setdefault(key,[]).append(i[key])
    return dic

def foo2(x):
    string=''
    for i in x:
        string+=i+' '
        
    return string

def open_pkl(path="data.pkl"):
    return pd.read_pickle(path)

def make_csv(input_path,output_path):
    pkl = open_pkl(input_path)
    df=pd.DataFrame()
    df['name']=pkl['name']
    df[['latitude','longtitude']]=pkl['coordinates'].apply(pd.Series)
    df[['address1','address2','address3','city','zip_code','country','state','display_address']]=pkl['location'].apply(pd.Series)
    df[['display_address']]=df['display_address'].apply(foo2)
    df[['alias,title']]=pkl['categories'].apply(foo)
    df[['alias','title']]=df['alias,title'].apply(pd.Series)

    df=df[['name','title','latitude','longtitude','address1','address2','address3','city','zip_code','country','state','display_address']]
    df=df.drop_duplicates(['name','latitude','longtitude','address1','address2','address3','city','zip_code','country','state','display_address'])
    df.to_csv(output_path, index=False)


In [21]:
make_csv("data.pkl","data.csv")

,name,title,latitude,longtitude,address1,address2,address3,city,zip_code,country,state,display_address
0,Wissahickon Bar & Bistro,"[Burgers, Bars]",40.017109,-75.210449,5109 Rochelle Ave,,None,Philadelphia,19128,US,PA,"5109 Rochelle Ave Philadelphia, PA 19128"
1,DJ Kitchen,"[Chinese, Bubble Tea]",40.007287,-75.212717,4040 City Ave,None,None,Philadelphia,19131,US,PA,"4040 City Ave Philadelphia, PA 19131"
2,The Rook Manayunk,"[Breakfast & Brunch, Juice Bars & Smoothies, S...",40.021691,-75.215963,4001 Cresson St,,None,Philadelphia,19127,US,PA,"4001 Cresson St Philadelphia, PA 19127"
3,Unity Taqueria,"[Tacos, Tex-Mex]",40.022452,-75.209024,5420 Ridge Ave,None,,Philadelphia,19128,US,PA,"5420 Ridge Ave Philadelphia, PA 19128"
4,Deke's Bar-B-Que,[Barbeque],40.015274,-75.207567,4901 Ridge Ave,,,Philadelphia,19128,US,PA,"4901 Ridge Ave Philadelphia, PA 19128"
...,...,...,...,...,...,...,...,...,...,...,...,...
140543,Michael's Trattoria,"[Italian, Pizza]",41.454051,-72.817450,344 Center St,,,Wallingford,06492,US,CT,"344 Center St Wallingford, CT 06492"
140544,Cheeks Chicken and Waffles,"[Waffles, Chicken Shop, Chicken Wings]",41.447960,-72.802400,822 E Center St,Ste D,None,Wallingford,06492,US,CT,"822 E Center St Ste D Wallingford, CT 06492"
140545,Nardelli's Grinder Shoppe,"[Delis, Sandwiches]",41.528088,-72.773706,896 E Main St,,,Meriden,06450,US,CT,"896 E Main St Meriden, CT 06450"
140546,The Local NY Style Pizzaria,[Pizza],41.526197,-72.754330,1448 E Main St,,None,Meriden,06450,US,CT,"1448 E Main St Meriden, CT 06450"


# Scrapy

In [ ]:
fetch('https://geographic.org/streetview/usa/')
states=response.css('ul')
states.css('a::text').getall()
states.css('a').attrib['href']

In [30]:
a='https://geographic.org/streetview/usa/ca/los_angeles/los_angeles.html'
a[::-1].find('/')
a[:-16]

'https://geographic.org/streetview/usa/ca/los_angeles/'

# Making city.txt

In [1]:
import pandas as pd
import argparse

In [16]:
df = pd.read_csv("street names/street.csv")
df

,state,county,city,streets
0,Washington,Yakima,White Swan,"1st Street,2nd Street,A Street,Abella Lane,B S..."
1,Washington,Yakima,Zillah,"1st Avenue,1st Street,2nd Street,4th Street,5t..."
2,Rhode Island,Washington,Wyoming,"1st Avenue,Acorn Trail,Aldrich Street,Baker Pi..."
3,South Carolina,Williamsburg,Nesmith,"Abbie Lane,Ash Drive,Battery Park Road,Battle ..."
4,South Carolina,Williamsburg,Salters,"A B Smith Road,Agriculture Avenue,Bamboo Loop,..."
...,...,...,...,...
29154,Colorado,Conejos,Romeo,"1st Avenue,2nd Avenue,3rd Avenue,4th Avenue,Al..."
29155,Colorado,Conejos,La Jara,"15G Road,15S Road,19 Road,1st Street,2nd Avenu..."
29156,Colorado,Conejos,Antonito,"10th Avenue, East,10th Avenue, West,11th Avenu..."
29157,Colorado,Conejos,Conejos,"County Road 13,County Road G.5,County Road G6,..."


In [27]:
states=list(set(df['state']))
df[df['state']=='Missouri']

,state,county,city,streets
11294,Missouri,Ralls,Saverton,"Saverton Drive,Saverton West Drive,Sherman Tra..."
11295,Missouri,Ralls,Perry,"Adams Trail,Alf Street,Arizona Lane,Boulware B..."
11296,Missouri,Ralls,New London,"1st Street,2nd Street,3rd Street,5th Street,6t..."
11302,Missouri,Ralls,Center,"Ash Trail,Asher Bridge Trail,Aspen Trail,Black..."
11303,Missouri,Worth,Sheridan,"110th Street,115 Th Trail,120th Street,130th S..."
...,...,...,...,...
15365,Missouri,Carroll,Bogard,"1st Street,2nd Street,3rd Street,4th Street,5t..."
15366,Missouri,Carter,Grandin,"2nd Street,3rd Street,5th Street,6th Street,7t..."
15367,Missouri,Carter,Fremont,"2nd Street,3rd Street,4th Street,Ash Street,Co..."
15368,Missouri,Carter,Ellsinore,"Arnold Avenue,Ball Park Road,Birch Street,Brya..."


In [34]:
def get_states(df):
    return list(set(df['state']))

def get_cities(state, df):
    df1=df[df['state']==states]
    return list(set(df1['city']))

def get_streets(city,df):
    street_col = df[df['city'] == city]['streets']
    street_col = list(street_col)
    out = []
    for i in street_col:
        out.extend(i.split(','))
    
    return list(set(out))

def make_streets(df,output):
    states=get_states(df)
    for state in states:
        cities=get_cities(state, df)
        for city in cities:
            streets = get_cities(city, df)
            with open(output+state+', '+city+'.txt', 'w') as f:
                for street in streets:
                    f.write(street+'\n')


In [28]:
df1=df[df['state']=='Missouri']
df1
df1[df1['city'] == "Perry"]['streets']

11295    Adams Trail,Alf Street,Arizona Lane,Boulware B...
Name: streets, dtype: object

In [32]:
streets=get_streets("Perry", df[df['state']=='Missouri'])
with open('street names/test.txt','w') as f:
    for street in streets:
        f.write(street+'\n')


In [42]:
f1 = lambda x: ', '.join(x.dropna())
#alternative for join only strings
# f1 = lambda x: ', '.join([y for y in x if isinstance(y, str)])
f2 = lambda x: [z for y in x for z in y]
d = dict.fromkeys(df.columns.difference(['city','streets']), f1)
d['streets'] = f2 

df2 = df.groupby('city', as_index=False).agg(d)
df2

TypeError: 'float' object is not iterable

In [43]:
aggregation_functions = {'streets':'sum'}
df2 = df.groupby(df['city']).aggregate(aggregation_functions)
df2


TypeError: unsupported operand type(s) for +: 'int' and 'str'

# Test

In [29]:
streets = read_street()
data=read_data().to_dict(orient='records')
log_street=[streets[1942],streets[2635],streets[2636],streets[3819],streets[3870]]
log_street

['Maris St', 'Pilgrim Ln', 'Pilling St', 'Welton St', 'William Pl']

In [30]:
log_street=['Maris St', 'Pilgrim Ln', 'Pilling St', 'Welton St', 'William Place']
term='restaurants'
location='Philadelphia, Pa'

for (i, street) in tqdm(enumerate(log_street)): 
            address = street +' '+ location
            try:
                data.extend(query_api(term, address).get('businesses'))
            except:
                df = pd.DataFrame(data)
                df.to_pickle(data_path)
                traceback.print_exc()
                sys.exit()
        
print("-------------------SAVING DATA---------------------")
df = pd.DataFrame(data)
df.to_pickle('data.pkl')
print("                       DONE                        ")

5it [00:06,  1.27s/it]


-------------------SAVING DATA---------------------


NameError: name 'data_path' is not defined

In [3]:
term='restaurants'
location='Philadelphia, Pa'
r=query_api(term, location)

In [4]:
r.get('businesses')[0]

{'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'dist

In [136]:
import time
a=[]
a.extend([])
start=time.time()
for i in range(10000):
    a.append(list(range(500)))
end=time.time()
print(end -start)

0.1197962760925293


In [135]:
import time
a=[]
start=time.time()
for i in range(10000):
    a.extend(list(range(500)))
end=time.time()
print(end -start)

0.08768534660339355


In [25]:
a= { 'alias': 'barbuzzo-philadelphia',
  'categories': [ {'alias': 'mediterranean', 'title': 'Mediterranean'},
                  {'alias': 'pizza', 'title': 'Pizza'},
                  {'alias': 'italian', 'title': 'Italian'}],
  'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
  'display_phone': '(215) 546-9300',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 1,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 2,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 3,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1600'},
                         { 'day': 4,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 5,
                           'end': '2200',
                           'is_overnight': False,
                           'start': '1200'},
                         { 'day': 6,
                           'end': '2100',
                           'is_overnight': False,
                           'start': '1200'}]}],
  'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
  'is_claimed': True,
  'is_closed': False,
  'location': { 'address1': '110 S 13th St',
                'address2': '',
                'address3': '',
                'city': 'Philadelphia',
                'country': 'US',
                'cross_streets': '',
                'display_address': ['110 S 13th St', 'Philadelphia, PA 19107'],
                'state': 'PA',
                'zip_code': '19107'},
  'name': 'Barbuzzo',
  'phone': '+12155469300',
  'photos': [ 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/dn5Pfj75I1fWZ2LqZ0nJ4Q/o.jpg',
              'https://s3-media1.fl.yelpcdn.com/bphoto/3hdgyod-3e5eW4rfhvVh0A/o.jpg'],
  'price': '$$',
  'rating': 4.5,
  'review_count': 2772,
  'transactions': ['pickup', 'delivery'],
  'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=LYlilwjYqjn0f0LIANnrQg'}

b= {'id': '6ajnOk0GcY9xbb5Ocaw8Gw',
 'alias': 'barbuzzo-philadelphia',
 'name': 'Barbuzzo',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/88mJSIDPi3JUsCK2FsVJqg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/barbuzzo-philadelphia?adjust_creative=LYlilwjYqjn0f0LIANnrQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LYlilwjYqjn0f0LIANnrQg',
 'review_count': 2772,
 'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.94999, 'longitude': -75.16216},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '110 S 13th St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19107',
  'country': 'US',
  'state': 'PA',
  'display_address': ['110 S 13th St', 'Philadelphia, PA 19107']},
 'phone': '+12155469300',
 'display_phone': '(215) 546-9300',
 'distance': 877.931789849032}